In [1]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('Data/listings.csv')

In [3]:
import re
import string

# Fill missing values with empty strings 
df['summary'] = df['summary'].fillna('')
df['space'] = df['space'].fillna('')
df['neighborhood_overview'] = df['neighborhood_overview'].fillna('')
df['notes'] = df['notes'].fillna('')
df['transit'] = df['transit'].fillna('')

# Creating a new column that combines all the columns with descriptions of the listings
df['content'] = df['name'] + ' ' + df['summary'] + ' ' + df['space'] + ' ' + df['description'] + ' ' + df['neighborhood_overview'] + ' ' + df['notes'] + ' ' + df['transit']


In [4]:
# Removing contractions to standardize text 
import contractions
expanded_words = []   
for word in df['content']:
# Using contractions.fix to expand the shortened words
  expanded_words.append(contractions.fix(word))     
df['content'] = expanded_words


In [5]:
# Creating a function that will preprocess and standardize the text for NLP 
def preprocess(text):
    ''' This function takes text to preprocess and standardize it for the purpose of NLP '''
 
    # Lowercase the text 
    text = text.str.lower()
    
    # Remove punctuations using string module with regular expression
    text = text.apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

    # Remove word and digits 
    #text = text.apply(lambda x: re.sub('W*dw*','', x))

    # Split string by whitespace and remoce extra whitespaces 
    text = text.apply(lambda x: x.split())

    # Remove stopwords 
    import nltk
    stopwords = nltk.corpus.stopwords.words('english')
    text = text.apply(lambda x: [word for word in x if word not in stopwords])

    # Lemmatize text to group together words with the same root or lemma
    from nltk.stem.wordnet import WordNetLemmatizer
    lemmatizer = nltk.stem.WordNetLemmatizer()
    text = text.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

    text = text.apply(lambda x: ' '.join(x))

    return text

In [6]:
df['content'] = preprocess(df['content'])

In [7]:
df['price'] = df['price'].replace('[\$,]','',regex=True).astype(float).astype(int)
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].median())
df['bedrooms'] = df['bedrooms'].astype(int)

In [17]:
df['neighbourhood_group_cleansed'].isna().sum()

0

In [18]:
df['room_type'].isna().sum()

0

In [19]:
df['bedrooms'].isna().sum()

0

In [20]:
df['accommodates'].isna().sum()

0

In [21]:
df['cancellation_policy'].isna().sum()

0

In [23]:
df.to_csv('Data/app_data.csv')

## Vectorize Text and Recommend

In [10]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 3), lowercase = False)
tfidf_matrix = tfidf.fit_transform(df['content'])

In [15]:
def recommend(content):
	similarity = linear_kernel(tfidf.transform(np.array([content])), tfidf_matrix)
	nums = np.argsort(similarity[0])[-5:]
	return df[['name', 'price', 'listing_url']].loc[nums]

In [16]:
# Recommend listings that offer free parking and return their name, price and url 
recommend('free parking')

,name,price,listing_url
1632,"Private Bedroom, Bathroom,& Parking",66,https://www.airbnb.com/rooms/8817033
572,Cozy Capitol Hill 1 BD Apartment,90,https://www.airbnb.com/rooms/10035644
416,Sweet+Cozy+Safe Studio in Seattle,60,https://www.airbnb.com/rooms/9866461
349,Wallingford 2 Bedroom Cozy Craftman,150,https://www.airbnb.com/rooms/1940787
210,Cozy Apartment near Bumbershoot,130,https://www.airbnb.com/rooms/3925573


In [40]:
df[(df.accommodates == 3) & (df.price <= 255) & (df.cancellation_policy == 'flexible') & (df.neighbourhood_group_cleansed == 'Queen Anne') 
& (df.room_type == 'Entire home/apt')] 

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,content
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,,A charming apartment that sits atop Queen Anne...,none,,,,NaN,NaN,https://a0.muscache.com/ac/pictures/94146944/6...,NaN,9851441,https://www.airbnb.com/users/show/9851441,Emily,2013-11-06,"Seattle, Washington, United States",NaN,NaN,NaN,NaN,f,https://a2.muscache.com/ac/users/9851441/profi...,https://a2.muscache.com/ac/users/9851441/profi...,Queen Anne,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"8th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.638473,-122.369279,t,Apartment,Entire home/apt,3,1.0,0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",NaN,100,$650.00,"$2,300.00",NaN,NaN,1,$0.00,1,1125,6 months ago,t,0,0,0,143,2016-01-04,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN,queen anne chateau charming apartment sits ato...
271,8088447,https://www.airbnb.com/rooms/8088447,20160104002432,2016-01-04,Central 1-BR with Lake Union View,This pleasant and centrally located apartment ...,Our home matches a calm atmosphere with a conv...,This pleasant and centrally located apartment ...,none,The Westlake neighborhood's proximity to Lake ...,We list this space as housing up to three gues...,Our condo is close to several King County Metr...,https://a1.muscache.com/ac/pictures/105070835/...,https://a1.muscache.com/im/pictures/105070835/...,https://a1.muscache.com/ac/pictures/105070835/...,https://a1.muscache.com/ac/pictures/105070835/...,5992456,https://www.airbnb.com/users/show/5992456,Hannah,2013-04-18,"Seattle, Washington, United States","A Seattle transplant from Colorado, I spend my...",within a day,100%,100%,f,https://a1.muscache.com/ac/users/5992456/profi...,https://a1.muscache.com/ac/users/5992456/profi...,Queen Anne,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Crockett Street, Seattle, WA 98109, United States",Queen Anne,East Queen Anne,Queen Anne,Seattle,WA,98109,Seattle,"Seattle, WA",US,United States,47.638314,-122.342931,t,Apartment,Entire home/apt,3,1.0,1,1.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Buzzer/...",NaN,103,NaN,NaN,NaN,NaN,1,$0.00,1,1125,2 months ago,t,0,0,0,93,2016-01-04,1,2015-11-09,2015-11-09,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,flexible,f,f,1,0.53,central 1 br lake union view pleasant centrall...
273,9199982,https://www.airbnb.com/rooms/9199982,20160104002432,2016-01-04,lakeview in QueenAnne near Frem

In [34]:
# Listings with strict cancellation policy and daily price of $50
df[(df.cancellation_policy == 'strict') & (df.price == 50)][['name', 'price']]

,name,price
690,UW Hacker House - 2nd flr 2p U,50
781,UW Hacker House- 2nd flr 2p L,50
994,The Nest,50
2005,Room in Cute Craftsman Style House,50
2043,Cozy modern home in Greenlake,50
2192,Single Bdr @Seattle Urban Homestead,50
2218,Cozy room in diverse neighborhood,50
2230,Bedroom w/ Kitchenette & Bathroom,50
3727,Cozy Bedroom with Private Bathroom,50
